In [67]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline
from sklearn.preprocessing import StandardScaler, MinMaxScaler, OrdinalEncoder
from sklearn.compose import make_column_transformer
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Ridge
from sklearn.feature_selection import RFECV
from sklearn.model_selection import StratifiedKFold
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, LabelEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report, roc_auc_score

In [55]:
df= pd.read_csv("LasVegasTripAdvisorReviews-Dataset.csv", sep=';')
df = df.drop(['User country'], axis=1)

print(df.shape)

(504, 19)


In [56]:
categorical = list(df.select_dtypes(include=['object']).columns.values)
le = preprocessing.LabelEncoder()
# seasons in place of months
['Dec-Feb' 'Mar-May' 'Jun-Aug' 'Sep-Nov']
df['Period of stay'] = df['Period of stay'].map({'Dec-Feb':'winter', 'Mar-May':'spring', 'Jun-Aug' :'summer','Sep-Nov':'autumn'})

for i in range(0, len(categorical)):
  #  df[categorical[i]] = le.fit_transform(df[categorical[i]])
    print(df[categorical[i]].unique())

['winter' 'spring' 'summer' 'autumn']
['Friends' 'Business' 'Families' 'Solo' 'Couples']
['NO' 'YES']
['YES' 'NO']
['NO' 'YES']
['NO' 'YES']
['YES' 'NO']
['YES' 'NO']
['Circus Circus Hotel & Casino Las Vegas' 'Excalibur Hotel & Casino'
 'Monte Carlo Resort&Casino' 'Treasure Island- TI Hotel & Casino'
 'Tropicana Las Vegas - A Double Tree by Hilton Hotel' 'Caesars Palace'
 'The Cosmopolitan Las Vegas' 'The Palazzo Resort Hotel Casino'
 'Wynn Las Vegas' 'Trump International Hotel Las Vegas' 'The Cromwell'
 'Encore at wynn Las Vegas' 'Hilton Grand Vacations on the Boulevard'
 "Marriott's Grand Chateau" 'Tuscany Las Vegas Suites & Casino'
 'Hilton Grand Vacations at the Flamingo' 'Wyndham Grand Desert'
 'The Venetian Las Vegas Hotel' 'Bellagio Las Vegas' 'Paris Las Vegas'
 'The Westin las Vegas Hotel Casino & Spa']
['3' '4' '5' '4,5' '3,5']
['North America' 'Europe' 'Asia' 'Oceania' 'Africa' 'South America']
['January' 'February' 'March' 'April' 'May' 'June' 'July' 'August'
 'September' 'O

In [57]:
for i in range(0, len(categorical)):
    df[categorical[i]] = le.fit_transform(df[categorical[i]])
df.head()

,Nr. reviews,Nr. hotel reviews,Helpful votes,Score,Period of stay,Traveler type,Pool,Gym,Tennis court,Spa,Casino,Free internet,Hotel name,Hotel stars,Nr. rooms,User continent,Member years,Review month,Review weekday
0,11,4,13,5,3,3,0,1,0,0,1,1,2,0,3773,3,9,4,4
1,119,21,75,3,3,0,0,1,0,0,1,1,2,0,3773,3,3,4,0
2,36,9,25,5,1,2,0,1,0,0,1,1,2,0,3773,3,2,3,2
3,14,7,14,4,1,3,0,1,0,0,1,1,2,0,3773,2,6,3,0
4,5,5,2,4,1,4,0,1,0,0,1,1,2,0,3773,3,7,7,5


In [58]:
# fill biến không hợp lệ bằng median
df.loc[df[df['Member years']<0].index, 'Member years']= np.NaN
df['Member years'].fillna(df['Member years'].median(), inplace=True)

In [60]:
X = df.drop('Score', axis=1)
y = df['Score']

In [61]:
df['Score'].value_counts()

5    227
4    164
3     72
2     30
1     11
Name: Score, dtype: int64

In [83]:
## prepare train and test labels
from sklearn.model_selection import train_test_split
X = df.drop(['Score'], axis=1)
y = df['Score']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify = df['Score'])

### Run model

In [78]:
from sklearn.model_selection import cross_val_score
def print_score(clf, X_train, y_train, X_test, y_test, train=True):
    if train:
        pred = clf.predict(X_train)
        clf_report = pd.DataFrame(classification_report(y_train, pred, output_dict=True))
        print("Train Result:\n================================================")
        print(f"Accuracy Score: {accuracy_score(y_train, pred) * 100:.2f}%")
        print("_______________________________________________")
        print(f"CLASSIFICATION REPORT:\n{clf_report}")
        print("_______________________________________________")
        print(f"Confusion Matrix: \n {confusion_matrix(y_train, pred)}\n")
        print("_______________________________________________")
        scores = cross_val_score(clf, X, y, cv=10, scoring='accuracy')
        print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


In [79]:
from sklearn.tree import DecisionTreeClassifier

dt_clf = DecisionTreeClassifier(criterion='entropy', 
                                 max_depth=4, 
                                 random_state=42)
dt_clf.fit(X_train, y_train)
print_score(dt_clf, X_train, y_train, X_test, y_test, train=True)
print_score(dt_clf, X_train, y_train, X_test, y_test, train=False)

Train Result:
Accuracy Score: 54.50%
_______________________________________________
CLASSIFICATION REPORT:
                  1          2          3           4           5  accuracy  \
precision  0.285714   0.500000   0.800000    0.539683    0.544828  0.544974   
recall     0.250000   0.173913   0.148148    0.276423    0.929412  0.544974   
f1-score   0.266667   0.258065   0.250000    0.365591    0.686957  0.544974   
support    8.000000  23.000000  54.000000  123.000000  170.000000  0.544974   

            macro avg  weighted avg  
precision    0.534045      0.571395  
recall       0.355579      0.544974  
f1-score     0.365456      0.484971  
support    378.000000    378.000000  
_______________________________________________
Confusion Matrix: 
 [[  2   0   2   1   3]
 [  1   4   0   3  15]
 [  0   2   8  15  29]
 [  2   2   0  34  85]
 [  2   0   0  10 158]]

_______________________________________________
Accuracy: 0.37 (+/- 0.22)


In [81]:
from sklearn.naive_bayes import GaussianNB
nb =  GaussianNB(var_smoothing=0.9)
nb.fit(X_train, y_train)
print_score(nb, X_train, y_train, X_test, y_test, train=True)
print_score(nb, X_train, y_train, X_test, y_test, train=False)

Train Result:
Accuracy Score: 44.97%
_______________________________________________
CLASSIFICATION REPORT:
             1     2     3      4           5  accuracy   macro avg  \
precision  0.0   0.0   0.0    0.0    0.449735  0.449735    0.089947   
recall     0.0   0.0   0.0    0.0    1.000000  0.449735    0.200000   
f1-score   0.0   0.0   0.0    0.0    0.620438  0.449735    0.124088   
support    8.0  23.0  54.0  123.0  170.000000  0.449735  378.000000   

           weighted avg  
precision      0.202262  
recall         0.449735  
f1-score       0.279033  
support      378.000000  
_______________________________________________
Confusion Matrix: 
 [[  0   0   0   0   8]
 [  0   0   0   0  23]
 [  0   0   0   0  54]
 [  0   0   0   0 123]
 [  0   0   0   0 170]]

_______________________________________________
Accuracy: 0.45 (+/- 0.02)


In [82]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

lr_clf = LinearRegression()
lr_clf.fit(X_train, y_train)
y_pred = lr_clf.predict(X_train)
mae = mean_absolute_error(y_train, y_pred)
mse = mean_squared_error(y_train, y_pred)
r2 = r2_score(y_train, y_pred)
# print results
print(f"MAE: {mae:.4f}")
print(f"MSE: {mse:.4f}")
print(f"R2 Score: {r2:.4f}")


MAE: 0.7374
MSE: 0.8565
R2 Score: 0.1534
